In [ ]:
import pandas as pd
df = pd.read_csv("df_hua.csv")
df.set_index('datetime', inplace=True)
# For DataFrame
df = df.astype(float)

# Sort the DataFrame by datetime
df.sort_index(inplace=True)


# Train-test split based on timestamp
train_size = int(len(df) * 0.66)
train, test = df.iloc[:train_size], df.iloc[train_size:]

# Features and target
X_train, y_train = train.drop('target', axis=1), train['target']
X_test, y_test = test.drop('target', axis=1), test['target']

In [16]:
df

,solar_fore_de_MW,solar_fore_it_MW,wind_fore_de_MW,wind_fore_it_MW,temperature_fore_ch,temperature_fore_fr,temperature_fore_de,temperature_fore_it,dayofyear,cosdayofyear,...,lag_std_mwh_15,lag_std_mwh_16,lag_std_mwh_17,lag_std_mwh_18,lag_std_mwh_19,lag_std_mwh_20,lag_std_mwh_21,lag_std_mwh_22,lag_std_mwh_23,lag_std_mwh_24
datetime,,,,,,,,,,,,,,,,,,,,,
2019-01-02 00:00:00+00:00,0.0,0.0,36316.9588,2230.8105,4.014400,6.405500,4.115400,3.578600,2.0,0.999407,...,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
2019-01-02 01:00:00+00:00,0.0,0.0,35664.6531,2138.4336,3.601709,6.023531,3.856946,3.726132,2.0,0.999407,...,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
2019-01-02 02:00:00+00:00,0.0,0.0,35411.2512,2014.8346,2.987348,5.530419,3.568760,3.893099,2.0,0.999407,...,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
2019-01-02 03:00:00+00:00,0.0,0.0,33715.8108,1931.9359,2.304482,5.022102,3.293981,4.129664,2.0,0.999407,...,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
2019-01-02 04:00:00+00:00,0.0,0.0,33287.7672,1816.4149,1.686276,4.594518,3.075752,4.485993,2.0,0.999407,...,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31 18:00:00+00:00,0.0,0.0,36997.7200,1108.4000,8.530000,9.960000,10.790000,9.590000,365.0,1.000000,...,-1.0,-1.0,-1.0,-1.0,20.987931,21.745693,21.965012,22.448986,23.396290,23.582733
2021-12-31 19:00:00+00:00,0.0,0.0,35666.9300,1077.9700,8.000000,9.400000,10.630000,9.110000,365.0,1.000000,...,-1.0,-1.0,-1.0,-1.0,-1.000000,20.987931,21.745693,21.965012,22.448986,23.396290
2021-12-31 20:00:00+00:00,0.0,0.0,34383.8800,1048.2800,7.500000,8.880000,10.510000,8.670000,365.0,1.000000,...,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,20.987931,21.745693,21.965012,22.448986


In [23]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# Initialize the Linear Regression model
lr = LinearRegression()

# Fit the model on training data
lr.fit(X_train, y_train)

# Predict on the test data
y_pred = lr.predict(X_test)

# Calculate the MAE
mae = mean_absolute_error(y_test, y_pred)

print(f"Mean Absolute Error: {mae}")


Mean Absolute Error: 18.61833057067334


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)
    
# Check CUDA and set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
y_train_tensor = torch.FloatTensor(y_train.values).to(device)
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
y_test_tensor = torch.FloatTensor(y_test.values).to(device)

# Create DataLoader for mini-batch gradient descent
train_data = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

# Define DNN model with dropout layers
class DNN(nn.Module):
    def __init__(self, input_dim):
        super(DNN, self).__init__()
        self.layer1 = nn.Linear(input_dim, 128)
        self.layer2 = nn.Linear(128, 32)
        self.dropout = nn.Dropout(0.2)
        self.layer3 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = self.dropout(x)
        x = torch.relu(self.layer2(x))
        x = self.dropout(x)
        x = self.layer3(x)
        return x

# Initialize model, loss, and optimizer
input_dim = X_train.shape[1]
model = DNN(input_dim).to(device)
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Early stopping initialization
min_val_loss = float('inf')
patience = 15
counter = 0

# Train the model
epochs = 100
for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()
        y_pred = model(batch_x)
        loss = criterion(y_pred.squeeze(), batch_y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    # Early stopping based on validation loss
    model.eval()
    with torch.no_grad():
        val_pred = model(X_test_tensor)
        val_loss = criterion(val_pred.squeeze(), y_test_tensor)
        if val_loss < min_val_loss:
            min_val_loss = val_loss
            counter = 0  # reset counter
        else:
            counter += 1  # increment counter
            if counter >= patience:
                print("Early stopping triggered")
                break

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {epoch_loss/len(train_loader):.4f}, Val Loss: {val_loss:.4f}")

# Evaluate the model
model.eval()
with torch.no_grad():
    y_test_pred = model(X_test_tensor).cpu().numpy()
    mae = mean_absolute_error(y_test, y_test_pred)
    print(f"Mean Absolute Error: {mae}")




Epoch [10/100], Train Loss: 21.5681, Val Loss: 17.5697


In [ ]:
# Save the model and scaler
torch.save(model.state_dict(), '../results_group1/model.pth')
joblib.dump(scaler, '../results_group1/scaler.pkl')


# STOP

In [ ]:
import pandas as pd



In [ ]:
# import lightgbm as lgb
# from sklearn.metrics import mean_squared_error, mean_absolute_error
# params = {
#     'objective': 'regression',
#     'metric': 'mae',
#     'boosting_type': 'gbdt',
#     'device': 'gpu',
#     'random_state': 42,
#     'gpu_platform_id' : 0,
#     'gpu_device_id': 0
# }

# # Create LightGBM data containers
# train_data = lgb.Dataset(X_train, label=y_train)
# test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)



# # Train the model
# num_round = 50
# bst = lgb.train(params, train_data, num_round, valid_sets=[test_data])

# # Make predictions
# y_pred = bst.predict(X_test)

# # Evaluate the model
# mse = mean_squared_error(y_test, y_pred)
# rmse = np.sqrt(mse)
# mae = mean_absolute_error(y_test, y_pred)

# print(f"RMSE: {rmse}")
# print(f"MAE: {mae}")

In [ ]:
import shap

# Initialize JavaScript for SHAP plots
shap.initjs()

# Create SHAP explainer
explainer = shap.Explainer(model)

# Calculate SHAP values for test data
shap_values = explainer(X_test)

# Plot SHAP summary
shap.summary_plot(shap_values, X_test)


In [ ]:
# X_train_tensor[:10]